In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-05 17:18:27.994319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 17:18:28.950534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 1000): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = DeepWordBugGao2018

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:1
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": default
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 14 / 10 / 1 / 25:   2%|▎         | 25/1000 [02:16<1:29:01,  5.48s/it]

[Succeeded / Failed / Skipped / Total] 26 / 21 / 3 / 50:   5%|▌         | 50/1000 [04:23<1:23:21,  5.26s/it]

[Succeeded / Failed / Skipped / Total] 40 / 31 / 4 / 75:   8%|▊         | 75/1000 [06:19<1:17:59,  5.06s/it]

[Succeeded / Failed / Skipped / Total] 55 / 38 / 7 / 100:  10%|█         | 100/1000 [08:00<1:12:04,  4.80s/it]

...

[Succeeded / Failed / Skipped / Total] 564 / 347 / 39 / 950:  95%|█████████▌| 950/1000 [1:17:41<04:05,  4.91s/it]

[Succeeded / Failed / Skipped / Total] 576 / 359 / 40 / 975:  98%|█████████▊| 975/1000 [1:19:40<02:02,  4.90s/it]

[Succeeded / Failed / Skipped / Total] 588 / 371 / 41 / 1000: 100%|██████████| 1000/1000 [1:21:59<00:00,  4.92s/it]

[Succeeded / Failed / Skipped / Total] 588 / 371 / 41 / 1000: 100%|██████████| 1000/1000 [1:22:00<00:00,  4.92s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 588    |
| Number of failed attacks:     | 371    |
| Number of skipped attacks:    | 41     |
| Original accuracy:            | 95.9%  |
| Accuracy under attack:        | 37.1%  |
| Attack success rate:          | 61.31% |
| Average perturbed word %:     | 25.12% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 100.35 |
+-------------------------------+--------+
The above are results for DeepWordBugGao2018_ag-news1000_mp0.3_nv11_default.
